In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

In [ ]:
def get_text_excluding_children(driver, element):
    return driver.execute_script("""
    return jQuery(arguments[0]).contents().filter(function() {
        return this.nodeType == Node.TEXT_NODE;
    }).text();
    """, element)

def driver_instance():
    driver = webdriver.Firefox()
    driver.get("https://volt.lider-bet.com/?v=102")
    return driver

def get_last_volt(driver):
    element = driver.find_elements(By.XPATH, '//div[contains(@class, "busted")]/span')[0]
    return get_text_excluding_children(driver, element)[:-1]

def get_current_bets(driver):
    current_bets = driver.find_elements(By.XPATH, '//tr[contains(@class, "current-bet")]/td[contains(@class, "td-bet")]')
    return [get_text_excluding_children(driver, current_bet) for current_bet in current_bets]

def get_win_bets(driver):
    win_bets = driver.find_elements(By.XPATH, '//tr[contains(@class, "win-bet")]/td[contains(@class, "td-bet")]')
    return [get_text_excluding_children(driver, win_bet) for win_bet in win_bets]

def get_win_nums(driver):
    win_nums = driver.find_elements(By.XPATH, '//tr[contains(@class, "win-bet")]/td[contains(@class, "td-num")]')
    return [get_text_excluding_children(driver, win_num) for win_num in win_nums]

def get_last_cashes(driver, cash_count = 70):
    cashes = driver.find_elements(By.XPATH, '//td[contains(@class, "td-cash")]/span[contains(@class, "num-volt")]')
    return [get_text_excluding_children(driver, cash)[:-1] for cash in cashes[:cash_count]]

def wait_game_end(driver, print_progress = False):
    if print_progress:
        print("Waiting for bet accepting...")
    
    WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'accepting-bets') and contains(@class, 'active')]"))
    )
    if print_progress:
        print("Bet accepting started, now waiting for bust")
    
    WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'busted') and contains(@class, 'active')]"))
    )
    if print_progress:
        print("busted!\n\n")

In [ ]:
def append_one_game_info(driver, file_name = "game_info"):
    wait_game_end(driver)
    with open(file_name, 'a') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(["last_volt" ,get_last_volt(driver)])
        wr.writerow(["current_bets"] + get_current_bets(driver))
        wr.writerow(["win_bets"] + get_win_bets(driver))
        wr.writerow(["win_nums"] + get_win_nums(driver))
        wr.writerow(["last_cashes"] + get_last_cashes(driver))

def append_n_game_info(driver, n = -1, file_name = "game_info"):
    if n != -1:
        for i in range(0, n):
            append_one_game_info(driver, file_name)
    else:
        while True:
            append_one_game_info(driver, file_name)

In [ ]:
driver = driver_instance()